http://www.sports-reference.com/cbb/postseason/1939-ncaa.html

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import element
from time import sleep
import pandas as pd


def get_page_data(year):
    url = 'http://www.sports-reference.com/cbb/postseason/'+str(year)+'-ncaa.html'
    page = urlopen(url)
    return BeautifulSoup(page, 'html.parser')

def tidy_location(locStr):
    if locStr.startswith('at '):
        return locStr[len('at '):]

In [42]:
def get_bracket():
    # this works for 84 first round bye and 85 and later 64+ team bracket ignoring play-in games
    years = range(2016, 1938, -1)
    #TODO fix years earlier than 1985
    bracket = []
    for year in years: 
        soup = get_page_data(year)
        region_tags = soup.find_all("div", attrs={"data-controls": "#brackets"})[0].select('div a')           
        #print(year)
        for region_tag in region_tags:        
            region = str(region_tag.string).lower()                           
            if region == 'final four':
                region_div = soup.find('div', {'id':'national'})
            else:
                search = {'id': region} 
                region_div = soup.find('div',**search)            
            if region_div is not None:
                bracket_rounds = region_div.div
                div_rounds = bracket_rounds.find_all('div', {'class':'round'})                
                bracket_round = 1
                for div_round in div_rounds:
                    #print("round, ", bracket_round)
                    if isinstance(div_round, element.Tag ):
                        #print(len(div_round))
                        for gm in div_round:
                            if isinstance(gm, element.Tag ):
                                #print(gm)
                                #print("len contents: ", len(gm.contents))
                                #print()
                                if (len(gm.contents) >= 9):
                                    #print(gm)
                                    #print()
                                    seeds_loc = gm.find_all('span')
                                    if len(seeds_loc) > 2:                                        
                                        winner = gm.find('div', {'class':'winner'})
                                        #if winner is None:
                                        #    print(winner)
                                        #else:
                                        winner_seed = winner.find('span').string                                        
                                        seed1 = seeds_loc[0].string                                        
                                        seed2 = seeds_loc[1].string                                        
                                        currLoc = tidy_location(seeds_loc[2].string)        
                                        alist = gm.select('div a')                                        
                                        team1 = alist[0].string
                                        #if len(alist) > 2:
                                        team2 = alist[2].string
                                        #else:
                                        #    print(alist)
                                        bracketRow = [year, region, bracket_round, seed1, team1, currLoc, seed2, team2, winner_seed]
                                        #print(bracketRow)
                                        bracket.append(bracketRow)
                        bracket_round += 1
        sleep(0.5)
    return pd.DataFrame(bracket, columns=['Year', 'Region', 'Round', 'Seed', 'Team', 'Location', 'OpponentSeed', 'Opponent', 'WinningSeed']) 

    
df = get_bracket()
#df
df.to_csv('ncaa.csv')

In [43]:
bracket_2017_and_2016 = df.head(64)
sample = bracket_2017_and_2016.head(8)

In [44]:
#sample_group = sample.groupby(['Team Name', 'Opponent'])
bracket_2017_and_2016

,Year,Region,Round,Seed,Team,Location,OpponentSeed,Opponent,WinningSeed
0,2016,east,1,1,UNC,"Raleigh, NC",16,Florida Gulf Coast,1
1,2016,east,1,8,USC,"Raleigh, NC",9,Providence,9
2,2016,east,1,5,Indiana,"Des Moines, IA",12,Chattanooga,5
3,2016,east,1,4,Kentucky,"Des Moines, IA",13,Stony Brook,4
4,2016,east,1,6,Notre Dame,"Brooklyn, NY",11,Michigan,6
5,2016,east,1,3,West Virginia,"Brooklyn, NY",14,Stephen F. Austin,14
6,2016,east,1,7,Wisconsin,"St. Louis, MO",10,Pitt,7
7,2016,east,1,2,Xavier,"St. Louis, MO",15,Weber State,2
8,2016,east,2,1,UNC,"Raleigh, NC",9,Providence,1
9,2016,east,2,5,Indiana,"Des Moines, IA",4,Kentucky,5


In [70]:
df1 = df.loc[(df['Round'] == 1)]
df1_teams = df1.loc[:, ['Team','Opponent']]
#df1_teams
#TODO get the years for each team's appearances?
um1 = df1_teams[(df1_teams.Team == 'Miami (FL)') | (df1.Opponent == 'Miami (FL)')]
uf1 = df1_teams[(df1_teams.Team == 'Florida') | (df1.Opponent == 'Florida')]
fsu1 = df1_teams[(df1_teams.Team == 'Florida State') | (df1.Opponent == 'Florida State')]
fgcu1 = df1_teams[(df1_teams.Team == 'Florida Gulf Coast') | (df1.Opponent == 'Florida Gulf Coast')]
ucf1 = df1_teams[(df1_teams.Team == 'UCF') | (df1.Opponent == 'UCF')]
#print("Miami", len(um1.index), "Florida", len(uf1.index), "FSU", len(fsu1.index))
#print("FL Gulf Coast", len(fgcu1.index), "UCF", len(ucf1.index))
#TODO get unique values for all teams ever, a count of them etc 
#df_years = df.groupby('Year')
#list(df_years)

#TODO is this an interesting side note to get random stats on each team grouped by year?  
#TODO do we need to export to a file?
df['Year'].groupby(df['Team']).count()

Team
Alabama                         21
Alabama-Birmingham               9
Alcorn State                     1
Appalachian State                1
Arizona                         52
Arizona State                   12
Arkansas                        33
Arkansas-Little Rock             1
Auburn                          14
Austin Peay                      2
BYU                             21
Ball State                       1
Baylor                          12
Boston College                  21
Boston University                1
Bowling Green State              2
Bradley                         11
Brown                            1
Butler                          18
CCNY                             2
Cal State Fullerton              1
Canisius                         3
Central Michigan                 2
Charlotte                       11
Chattanooga                      2
Cincinnati                      43
Clemson                         11
Cleveland State                  1
College of Char